# Generating_Images

### Headers

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]='0'
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
#import seaborn as sns
#import warnings
#import csv
import gc
from time import time
import logging
#from scipy.io import loadmat
#from select_win import select_win,select_windows
import load_data as ldfile
#import create_model as cmodel
import glob
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

tf.device('/device:GPU:0')

#from scipy.signal import butter, sosfilt, sosfilt_zi, sosfiltfilt, lfilter, lfilter_zi, filtfilt, sosfreqz, resample
#from utils import hamilton_detector, christov_detector, findpeaks, engzee_detector
from ecg_detectors.ecgdetectors import Detectors, MWA, panPeakDetect, searchBack

np.random.seed(354)
#sns.set()
#warnings.filterwarnings('ignore')
#logging.getLogger("tensorflow").setLevel(logging.ERROR)
tf.random.set_seed(1342) # Set seed for reproducibility

if tf.test.gpu_device_name(): 
    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))
else:
   print("Please install GPU version of TF")


Default GPU Device:/device:GPU:0


### Loading Up Images from Dataset

In [3]:
t1 = time()
input_directory = '/dataset/PhysioNet2020/Training_WFDB' # Dataset on beastie 
train_data,train_labels_bin,val_data,val_labels_bin = ldfile.load_data2D_bin(input_directory) 
#train_dataset,val_dataset = ldfile.load_tf(train_data,train_labels_bin,val_data,val_labels_bin)
t2 = time()
print('Loading data in {} seconds.'.format(round(t2-t1)))


total number of files:  6877
Number of files for trainig:  5501
Number of files for validation:  687
Number of files for testing:  687
500 files were loaded for training!
1000 files were loaded for training!
1500 files were loaded for training!
2000 files were loaded for training!
2500 files were loaded for training!
3000 files were loaded for training!
3500 files were loaded for training!
4000 files were loaded for training!
4500 files were loaded for training!
5000 files were loaded for training!
5500 files were loaded for training!
499 files were loaded for validation!
Labels:
0. 164884008
1. 164889003
2. 164909002
3. 164931005
4. 270492004
5. 284470004
6. 426783006
7. 429622005
8. 59118001

number of test signals =  689

train_data type: <class 'list'>
train_labels  type: <class 'list'>
val_data  type: <class 'list'>
val_labels  type: <class 'list'>

train_data length 26828
train_labels length 26828
val_data length: 3532
val_labels length: 3532

train_data type: <class 'numpy.ndarr

In [4]:
train_data.shape

(26828, 12, 2500, 1)

### Plotting the Data from the dataset as an image

In [5]:
def create_plot(data1, i, dir_path):
    
    #plt.figure()
    #figure(num=None, figsize=(100,10), dpi = 100)
    
    #fig,lead=plt.subplots(12,1, figsize=(20,60)) #### ORIGINAL - FOR ALL 12 LEADS
    fig,lead=plt.subplots(4,1, figsize=(10,20), dpi = 100)
    #plt.gca().set_axis_off()
    
    plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, hspace = 0, wspace = 0) #deals with all the whitespace around the plots
    #plt.margins(0,0)

    lead[0].plot(data1[0],'black')
    lead[0].set_axis_off()
    #lead[1].plot(data1[1],'black')
    #lead[1].set_axis_off()
    #lead[2].plot(data1[2],'black')
    #lead[2].set_axis_off()
    #lead[3].plot(data1[3],'black')
    #lead[3].set_axis_off()
    lead[1].plot(data1[4],'black')
    lead[1].set_axis_off()
    #lead[5].plot(data1[5],'black')
    #lead[5].set_axis_off()
    lead[2].plot(data1[6],'black')
    lead[2].set_axis_off()
    #lead[7].plot(data1[7],'black')
    #lead[7].set_axis_off()
    #lead[8].plot(data1[8],'black')
    #lead[8].set_axis_off()
    #lead[9].plot(data1[9],'black')
    #lead[9].set_axis_off()
    #lead[10].plot(data1[10],'black')
    #lead[10].set_axis_off()
    lead[3].plot(data1[11],'black')
    lead[3].set_axis_off()
    
    file_name = 'img%d.png' %(i) 
    #fig.tight_layout()
    #plt.show()
    fig.savefig(os.path.join(dir_path,file_name), transparent = None, bbox_inches = 'tight', pad_inches = 0)
    plt.close()

STEPS:

1. Create an output folder (test to see if one exists)
2. Loop through 'train_data' -> plot images (do only 5000 at a time)

3. Save image into output folder
4. Take plotted image, open it and read it with imread (as an array item) 
5. append it to an array 'train_im'

6. Read through 'train_labels_bin' and copy section (5000 at a time) into 'train_lab'

7. Repeat process for 'val_data' and 'val_labels_bin'

8. Make train_dataset and val_dataset from data and labels combined.

### TRAINING DATA

#### **ERROR** - Separate 'train_batch' into 2 functions and break the loop for automating into two parts

In [6]:
def train_batch (train_data, start, end):
    
    # test block to test the function in step 1:
    dir_path = "train_outputs_v1"
    os.makedirs(dir_path, exist_ok=True)

    diff = end-start
    range_end = 1000+diff
    # test block to test the loop in step 2 & 3:
    for i,j in zip (range(start, end), range(10000,range_end)):  #the second range is for the file numbers you're gonna generate in the 
        test_1 = create_plot(train_data[i], j, dir_path)
    

    # test blcok to test saving the img in step 4 & 5:
    X_data = []
    files = sorted(glob.glob('train_outputs_v1/*.png')) #change temp_outputs to your temporary storage area for images created
    for myFile in files:
        image = cv2.imread (myFile)
        X_data.append (image)
    #train_im = np.asarray(X_data)
    
    #return train_im
    return X_data

In [7]:
# Tracking for Training Data:

#Train_data size: 26828
#single 'train_batch': 500
#number of batches to load: 53.656

t1 = time()

#train_d1 = [[]]
for i in range(0,6):
    length = 1000 # pick how big do you want your length to be
    start = i*length
    end = start+length
    print('round:', i, 'start:', start, 'end:', end)
    t_d = train_batch(train_data, start, end)
    print('converting into array for round:', i)
    if i==0:
        train_d = np.asarray(t_d)
        #print('starting to work')
    else:
        train_d1 = np.asarray(t_d)
        #print('is it working?')
        train_d = np.append(train_d, train_d1 , axis=0) 
    
    del t_d
    gc.collect()

print(train_d.shape)
    
np.save('train_v1.npy', train_d)
del train_d, train_d1
gc.collect()
    
t2 = time()
print('Loading data in {} seconds.'.format(round(t2-t1)))   

round: 0 start: 0 end: 1000
converting into array for round: 0
round: 1 start: 1000 end: 2000
converting into array for round: 1
round: 2 start: 2000 end: 3000
converting into array for round: 2
round: 3 start: 3000 end: 4000
converting into array for round: 3
round: 4 start: 4000 end: 5000
converting into array for round: 4
round: 5 start: 5000 end: 6000
converting into array for round: 5
(0,)
Loading data in 1 seconds.


In [ ]:
## DO NOT DO THIS UNTIL ALL 26828  are loaded in train_set_#.npy files
#del train_data
#gc.collect()

### VALIDATION DATA

In [8]:
def val_batch (val_data, start, end):
    

    # test block to test the function in step 1:
    dir_path = "val_outputs_v1"
    os.makedirs(dir_path, exist_ok=True)

    # test block to test the loop in step 2 & 3:
    diff = end-start
    range_end = 1000+diff
    for i,j in zip (range(0,1000), range(10000,range_end)): #the second range is for the file numbers you're gonna generate in the 
        test_1 = create_plot(val_data[i], j, dir_path)

    # test blcok to test saving the img in step 4 & 5:
    X_data = []
    files = sorted(glob.glob('val_outputs_v1/*.png')) #change temp_outputs to your temporary storage area for images created
    for myFile in files:
        
        image = cv2.imread (myFile)
        X_data.append (image)
    #val_im = np.asarray(X_data)

    
    #return val_im
    return X_data

In [9]:
# Tracking for Validation Data:

#Train_data size: 3532
#single 'train_batch': 500
#number of batches to load: ~6

t1 = time()

#train_d1 = [[]]
for i in range(0,3):
    length = 1000 # pick how big do you want your length to be
    start = i*length
    end = start+length
    print('round:', i, 'start:', start, 'end:', end)
    v_d = val_batch(val_data, start, end)
    print('converting into array for round:', i)
    if i==0:
        val_d = np.asarray(v_d)
    else:
        val_d1 = np.asarray(v_d)
        val_d = np.append(val_d, val_d1 , axis=0) 
    
    del v_d
    gc.collect()
    
print(val_d.shape)
    
np.save('val_v1.npy', val_d)
del val_d, val_d1
gc.collect()
    
t2 = time()
print('Loading data in {} seconds.'.format(round(t2-t1)))   

round: 0 start: 0 end: 100
converting into array for round: 0
round: 1 start: 100 end: 200
converting into array for round: 1
Loading data in 150 seconds.


In [2]:
## DO NOT DO THIS UNTIL ALL 26828  are loaded in train_set_#.npy files
#del val_data
#gc.collect()